In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append('../../')

In [25]:
from src.indexing import get_multivector_retriever, get_parent_child_splits
from src.generation import ROUTER_SYSTEM_PROMPT, ROUTER_PROMPT, QA_SYSTEM_PROMPT, QA_PROMPT, SUMMARY_SYSTEM_PROMPT, SUMMARY_PROMPT, FINAL_SUMMARY_SYSTEM_PROMPT, FINAL_SUMMARY_PROMPT, LLAMA_PROMPT_TEMPLATE, MIXTRAL_PROMPT_TEMPLATE
from src.generation import get_model, format_docs, get_rag_chain, get_router_chain, summarize
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate


from src.ingestion import load_pdf

import os
import chromadb
import uuid
import pickle

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from tqdm import tqdm

import pandas as pd

In [7]:
DATA_PATH = 'D:\Ahmed\saudi-rag-project\data'
RAW_DOCS_PATH = os.path.join(DATA_PATH, "raw")
CHROMA_PATH = os.path.join(DATA_PATH, "chroma")
INTERIM_DATA_PATH = os.path.join(DATA_PATH, "interim")

EMBEDDING_MODEL_NAMES = [
    "intfloat/multilingual-e5-small", 
    "intfloat/multilingual-e5-base", 
    "text-embedding-3-small", 
    "text-embedding-3-large",
    "text-embedding-ada-002"
 ]
 
MODEL_NAMES = ["meta-llama/Llama-3-8b-chat-hf", "meta-llama/Llama-3-70b-chat-hf", "mistralai/Mixtral-8x22B-Instruct-v0.1"]

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_12020\2388391618.py:1: SyntaxWarning: invalid escape sequence '\A'
  DATA_PATH = 'D:\Ahmed\saudi-rag-project\data'


In [8]:
persistent_client = chromadb.PersistentClient(path=CHROMA_PATH)

In [36]:
# Load summary retriever
embedding_model = "text-embedding-3-small"
collection_name = 'PS_ALL_text_embedding_3_small'
retriever = get_multivector_retriever(persistent_client, embedding_model, collection_name, DATA_PATH, k=5)

In [56]:
MODEL_NAME = MODEL_NAMES[1]

In [57]:
if "mistral" in MODEL_NAME:
    MODEL_TEMPLATE = MIXTRAL_PROMPT_TEMPLATE

elif "llama" in MODEL_NAME:
    MODEL_TEMPLATE = LLAMA_PROMPT_TEMPLATE

In [105]:
SUMMARY_SYSTEM_PROMPT = """You are an assistant for summarization and question answering tasks. \
Use the following context to write a comprehensive summary to fullfill the user query. \
If the context doesn't contain any relevant information, just say you don't know. \
Answer in Arabic only."""

FINAL_SUMMARY_SYSTEM_PROMPT = """You are an expert Arabic assistant for question-answering tasks. \
Use the following summaries to create a comprehensive answer to the provided query. \
Answer in Arabic only."""

In [106]:
router_prompt_template = PromptTemplate.from_template(MODEL_TEMPLATE.format(system_prompt=ROUTER_SYSTEM_PROMPT, user_message=ROUTER_PROMPT))
qa_prompt_template = PromptTemplate.from_template(MODEL_TEMPLATE.format(system_prompt=QA_SYSTEM_PROMPT, user_message=QA_PROMPT))
summarization_prompt_template = PromptTemplate.from_template(MODEL_TEMPLATE.format(system_prompt=SUMMARY_SYSTEM_PROMPT, user_message=SUMMARY_PROMPT))
final_summary_template = PromptTemplate.from_template(MODEL_TEMPLATE.format(system_prompt=FINAL_SUMMARY_SYSTEM_PROMPT, user_message=FINAL_SUMMARY_PROMPT))

llm = get_model(MODEL_NAME)

router_chain = get_router_chain(llm, router_prompt_template)
summarization_chain = get_rag_chain(llm, retriever, format_docs, summarization_prompt_template)

In [107]:
question = 'اعطني ملخص عن ابرز الأخبار للمجموعة'

In [108]:
print(router_chain.invoke(question))



SUMMARY


In [109]:
retrieved_docs = retriever.invoke(question)

In [110]:
retrieved_docs

[Document(page_content='أﺑﺮز ﻣﻼﻣﺢ أداء اﻷﻋﻤﺎل\nواﺻﻠﺖ ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ رﻳﺎدﺗﻬﺎ ﻣﻦ ﺧﻼل اﺳﺘﻤﺮاﻫﺎ ﻓﻲ ﺗﻌﺰﻳﺰ اﻟﺴﻮق اﻟﻤﺎﻟﻴﺔ اﻟﺴﻌﻮدﻳﺔ و إﻃﻼق ﻋﺪد ﻣﻦ\nاﻟﻤﺒﺎدرات وذﻟﻚ ﺑﻤﺎ ﻳﺘﻤﺎﺷﻰ ﻣﻊ اﺳﺘﺮاﺗﻴﺠﻴﺘﻬﺎ ﻟﺘﺴﺮﻳﻊ اﻟﻨﻤﻮ وﺗﻌﺰﻳﺰ اﻷداء اﻟﺘﺸﻐﻴﻠﻲ واﻟﻤﺎﻟﻲ وﺗﻨﻮﻳﻊ اﻟﺨﺪﻣﺎت ذات اﻷﺛﺮ\nاﻹﻳﺠﺎﺑﻲ ﻋﺒﺮ ﺗﻮﻓﻴﺮ اﻟﺤﻠﻮل اﻟﻤﺒﺘﻜﺮة واﻟﺘﻘﻨﻴﺎت وﺗﻮﺳﻴﻊ ﻗﺎﻋﺪة اﻟﻌﻤﻼء اﻟﻤﺤﻠﻴﺔ واﻹﻗﻠﻴﻤﻴﺔ واﻟﻌﺎﻟﻤﻴﺔ.\n• ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ وﺻﻨﺪوق اﻻﺳﺘﺜﻤﺎرات اﻟﻌﺎﻣﺔ ﻳﻌﻠﻨﺎن ﻋﻦ ﺗﺄﺳﻴﺲ ﺷﺮﻛﺔ ﺳﻮق اﻟﻜﺮﺑﻮن اﻟﻄﻮﻋﻲ\nاﻻﻗﻠﻴﻤﻴﺔ واﻟﺘﻲ ﺗﻬﺪف إﻟﻰ دﻋﻢ اﻟﺸﺮﻛﺎت واﻟﻘﻄﺎﻋﺎت ﻓﻲ اﻟﻤﻨﻄﻘﺔ ﻟﺘﻤﻜﻴﻨﻬﺎ ﻣﻦ اﻟﻮﺻﻮل إﻟﻰ اﻟﺤﻴﺎد اﻟﺼﻔﺮي\nﺑﺎﻹﺿﺎﻓﺔ إﻟﻰ ﺿﻤﺎن ﺷﺮاء أرﺻﺪة اﻟﻜﺮﺑﻮن ﻟﺘﺨﻔﻴﺾ اﻻﻧﺒﻌﺎﺛﺎت اﻟﻜﺮﺑﻮﻧﻴﺔ ﻓﻲ ﺳﻼﺳﻞ اﻟﻘﻴﻤﺔ .\n• اﻛﺘﻤﺎل أول إدراج ﻣﺰدوج وﻣﺘﺰاﻣﻦ ﺑﻴﻦ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ وﺳﻮق أﺑﻮﻇﺒﻲ ﻟﻸوراق اﻟﻤﺎﻟﻴﺔ .\n• ﺗﻮﻗﻴﻊ "واﻣﺾ" إﺣﺪى اﻟﺸﺮﻛﺎت اﻟﺘﺎﺑﻌﺔ ﻟﻤﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ اﺗﻔﺎﻗﻴﺔ ﺑﻴﻊ وﺷﺮاء ﻣﻊ ﺷﺮﻛﺔ "اﻟﻤﺒﺎدرات اﻟﺜﺎﻧﻴﺔ\nﻟﻼﺳﺘﺜﻤﺎر" ﻟﻼﺳﺘﺤﻮاذ ﻋﻠﻰ %51 ﻣﻦ رأس اﻟﻤﺎل اﻟﻤﺼﺪر ﻟﺸﺮﻛﺔ ﺷﺒﻜﺔ ﻣﺒﺎﺷﺮ اﻟﻤﺎﻟﻴﺔ "داﻳﺮﻛﺖ إف إ ن".\n• إﻃﻼق إﻃﺎر ﻋﻤﻞ ﺻﺎﻧﻊ اﻟﺴﻮق ﻷﺳﻮاق اﻷﺳﻬﻢ واﻟﻤﺸﺘﻘﺎت اﻟﻤﺎﻟﻴﺔ ﻛﺠﺰء ﻣﻦ ﺟﻬﻮد ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﻟﻀﻤﺎن ﺗﻮﻓﺮ\nاﻟﺴﻴﻮﻟﺔ وز

In [111]:
[question]*len(retrieved_docs)

['اعطني ملخص عن ابرز الأخبار للمجموعة',
 'اعطني ملخص عن ابرز الأخبار للمجموعة',
 'اعطني ملخص عن ابرز الأخبار للمجموعة',
 'اعطني ملخص عن ابرز الأخبار للمجموعة']

In [112]:
summary_answers = summarization_chain.batch({"question": [question]*len(retrieved_docs),
"context": [d.page_content for d in retrieved_docs]})

In [113]:
for a in summary_answers: print(a, '\n')



ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم م2022

أﺑﺮز ﻣﻼﻣﺢ أداء اﻟﺴﻮق اﻟﻤﺎﻟﻴﺔ اﻟﺴﻌﻮدﻳﺔ ﻓﻲ اﻟﻌﺎم م2022، ﻭﺗﺤﻘﻴﻖ اﻟﺴﻮق اﻟﻤﺎﻟﻴﺔ اﻟﺴﻌﻮدﻳﺔ اﻟﻤﺮﻛﺰ اﻟﺘﺎﺳﻊ ﻋﺎﻟﻤﻴﺎ ﻛﺄﻛﺒﺮ ﺳﻮق ﻣﺎﻟﻴﺔ ﻣﻦ ﻣﻦ ﺣﻴﺚ اﻟﻘﻴﻤﺔ اﻟﺴﻮﻗﻴﺔ.

وﺷﻬﺪت ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﻓﻲ اﻟﻌﺎم 2022 إﻧﺠﺎزا ﻗﻴﺎﺳﻴﺎ ﺑﺘﺴﺠﻴﻞ أﻛﺒﺮ ﻋﺪد ﻣﻦ ﻋﻤﻠﻴﺎت اﻹدراج ﻓﻲ ﻣﻨﻄﻘﺔ اﻟﺸﺮق اﻷوﺳﻂ، ﺑﻠﻐﺖ ﻗﻴﻤﺔ اﻟﻤﺘﻮﺳﻂ اﻟﻴﻮﻣﻲ ﻟﻘﻴﻤﺔ اﻷﺳﻬﻢ اﻟﻤﺘﺪاوﻟﺔ ﻓﻲ اﻟﺴﻮق اﻟﺮﺋﻴﺴﻴﺔ وﻧﻤﻮ - اﻟﺴﻮق اﻟﻤﻮازﻳﺔ 6.9 ﻣﻠﻴﺎر رﻳﺎل ﺳﻌﻮدي.

وأﺭﺗﻔﻊ ﻋﺪد اﻟﺸﺮﻛﺎت اﻟﻤﺪرﺟﺔ ﻓﻲ اﻟﺴﻮق اﻟﺮﺋﻴﺴﻴﺔ وﻧﻤﻮ - اﻟﺴﻮق اﻟﻤﻮازﻳﺔ إﻟﻰ 269 ﺷﺮﻛﺔ ﺑﻨﻬﺎﻳﺔ اﻟﻌﺎم اﻟﻤﺎﻟﻲ م2022، ﻣﻘﺎرﻧﺔ ﺑ 224 ﻓﻲ اﻟﻌﺎم اﻟﻤﺎﻟﻲ 2021 م.

وأﺿﺎف اﻟﺤﺼﺎن: "ﺧﻼل اﻟﻌﺎم م2022 ﻋﻤﻠﺖ اﻟﻤﺠﻤﻮﻋﺔ ﻋﻠﻰ ﺗﻘﺪﻳﻢ ﻋﺪد ﻣﻦ اﻟﺘﺤﺴﻴﻨﺎت ﻋﻠﻰ اﻟﺒﻨﻴﺔ اﻟﺘﺤﺘﻴﺔ ﻟﻠﺴﻮق اﻟﻤﺎﻟﻴﺔ اﻟﺴﻌﻮدﻳﺔ ﺑﻬﺪف اﺳﺘﻀﺎﻓﺔ ﻣﺠﻤﻮﻋﺔ ﻣﺘﻨﻮﻋﺔ ﻣﻦ اﻟﻤﺼﺪرﻳﻦ واﻟﻤﺴﺘﺜﻤﺮﻳﻦ". 



ﺍﻟﺨﻼﺻﺔ:

ﺗﻢ ﺗﻨﻈﻴﻢ ﻗﻄﺎﻋﺎت أﻋﻤﺎل اﻟﻤﺠﻤﻮﻋﺔ ﻋﻠﻰ أﺳﺎس اﻟﺨﺪﻣﺎت اﻟﻤﻘﺪﻣﺔ ﻓﻲ اﻟﻌﺎم 2022، ﻣﻊ ﻧﺨﻔﺎض إﻳﺮادات ﻗﻄﺎع أﺳﻮاق رأس اﻟﻤﺎل ﺑﻨﺴﺒﺔ 17% ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي، ﻟﺘﺒﻠﻎ 406.5 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي. ﻛﻤﺎ ﺗﻢ ﺗﻨﻈﻴﻢ إﻳﺮادات ﺧﺪﻣﺎت اﻟﺘﺪاول ﺑﻨﺴﺒﺔ 23.4% ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي، ﻟﺘﺒﻠﻎ 313.4 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي.

ﻓﻲ ﻣﺎ 

In [114]:
final_answer_chain = (
    final_summary_template
    | llm
    | StrOutputParser()
)

In [115]:
final_answer = final_answer_chain.invoke({"question": question, "context": "\n\n".join(summary_answers)})

In [116]:
print(summarization_chain.invoke(question))



ملخص للأخبار الرئيسية للمجموعة:

* أعلنت مجموعة تداول السعودية عن إطلاق شركة سوق الكربون الإقليمية والتي تهدف إلى دعم الشركات والقطاعات في المنطقة لتمكينها من الوصول إلى الحياد الصفري.
* أتمت المجموعة أول إدراج مزود ومتزامن بين تداول السعودية وسوق أبوظبي للأوراق المالية.
* وقعت المجموعة اتفاقية شراكة مع شركة "المبادرات الثانية للاستثمار" للاستحواذ على %51 من رأس المال المصدر لشركة شبكة مباشر المالية "دايركت إف إن".
* أطلقت المجموعة خدمة مقاصة إتفاقية إعادة الشراء.
* أعلنت المجموعة عن إطلاق إطار عمل لضمان توفير السيولة وزيادة كفاءة تحديد الأسعار تماشياً مع دعم نمو السوق المالي السعودي وتحقيق رؤية المملكة 2030.
* أطلقت المجموعة شركة مركزي مقاصة الأوراق المالية "مقاصة" خدمة مقاصة إتفاقية إعادة الشراء.


In [117]:
print(final_answer)



أبرز الأخبار للمجموعة في عام 2022:

أعلنت المجموعة عن نتائجها المالية السنوية لعام 2022، والتي شهدت تحقيق السوق المالية السعودية المركزي أعلى قيمة سوقية في التاريخ. وشهدت التداولات السعودية في عام 2022 إنجازا قياسيا بتسجيل أكبر عدد من عمليات الإدراج في منطقة الشرق الأوسط، بلغت قيمة المتوسط اليومي لقيمة الأسهم المتداولة في السوق الرئيسية ونمو السوق الموازية 6.9 مليار ريال سعودي.

وارتفع عدد الشركات المدرجة في السوق الرئيسية ونمو السوق الموازية إلى 269 شركة بنهاية عام 2022، مقارنة بـ 224 في عام 2021.

وأضاف الحصان: "خلال عام 2022 عملت المجموعة على تقديم عدد من التحسينات على البنية التحتية للسوق المالية السعودية بهدف استضافة مجموعة متنوعة من المصدرين والمستثمرين".

وأبرزت الخلاصة أن المجموعة قامت بتنظيم قطاعات أعمالها على أساس الخدمات المقدمة في عام 2022، مع انخفاض إيرادات قطاع أسواق رأس المال بنسبة 17% على أساس سنوي، لتبلغ 406.5 مليون ريال سعودي. كما قامت بتنظيم إيرادات خدمات التداول بنسبة 23.4% على أساس سنوي، لتبلغ 313.4 مليون ريال سعودي.

وفيما يتعلق بالمعلومات القطاعية، قامت بتنظيم 

In [ ]:
def summarize(question, retriever, summarization_chain, final_answer_chain, format_doc_fn):
    retrieved_docs = retriever.invoke(question)
    summary_answers = summarization_chain.batch([d.page_content for d in retrieved_docs])
    final_answer = final_answer.invoke({"question": question, "context": format_doc_fn(summary_answers)})
    return final_answer